#### Imports

In [23]:
import time
from openai import OpenAI
from Assistant import AssistantOpenAI
import logging
import os
import json
import gymnasium as gym
import random
import matplotlib.pyplot as plt
import os
import time
from git import Repo

import numpy as np
from collections import deque, namedtuple

# For visualization
import gymnasium.wrappers.record_video as record_video

#### Parameters

In [24]:
num_episodes = 200
ARCLABKEY_OPENAI = "sk-proj-DvHDR3hpgbm2r3kCA9jKT3BlbkFJL57ABXkfaWAIYKBxdhM6"
ARCLABKEY_OPENAI = "sk-proj-GvaDXazpibWA2M1I5Pu2T3BlbkFJxDuKlr9AcoVG98ctJZ7Q"

In [25]:
def store_code_in_file(code, filename):
    """ Store code in a file
    
    Args:
        code: str: code to store
        filename: str: filename to store code in
    
    Returns:
        None
    """
    with open(filename, 'w') as f:
        f.write(code)
        
store_code_in_file_schema = {
    "name": "store_code_in_file",
    "description": "Store code in a file",
    "parameters": {
        "type": "object",
        "properties": {
            "code": {
                "type": "string",
                "description": "The Python code to store."
            },
            "filename": {
                "type": "string",
                "description": "The filename to store the code in."
            }
        },
        "required": ["code", "filename"]
    }
}

TOOLS = [{"type": "function", "function": store_code_in_file_schema}]

### Entorno Lunnar Lander

In [26]:
env = gym.make('LunarLander-v2')
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

State shape:  (8,)
Number of actions:  4


In [27]:
agent_initial_code = """
def act(observation):
    '''
    The function that codifies the action to be taken in each instant of time.

    Args:
        observation (numpy.array):
            "description": "The state of the environment after the action is taken.",
            "positions": {  
                "0": "X position",
                "1": "Y position",
                "2": "X velocity",
                "3": "Y velocity",
                "4": "Angle",
                "5": "Angular velocity",
                "6": "Left contact sensor",
                "7": "Right contact sensor"
            },
            "min_values": [-1.5, -1.5, -5.0, -5.0, -3.14, -5.0, 0, 0],
            "max_values": [1.5, 1.5, 5.0, 5.0, 3.14, 5.0, 1, 1]

    Returns:
        Integer  : The action to be taken.
    '''
    return random.randint(0, 3)
"""

#### Mapeador de logs a JSON

In [28]:
log_description = {
    "description": "Log data for each step of the spacecraft landing environment.",
    "landing attempt": {
        "type": "integer",
        "description": "The episode number."
    },
    "logs": {
        "instant": {
            "type": "integer",
            "description": "The instant within the landing attempt where the current log is taken."
        },
        "action": {
            "type": "integer",
            "description": "The possible actions taken.",
            "options": {
                1 : "Switch off engines",
                2 : "Push left engine",
                3 : "Push both engines (upwards)",
                4 : "Push right engine"
            }
        },  
        "current status": {
            "type": "array",
            "description": "The state of the environment after the action is taken.",
            "positions": {  
                "0": "X position",
                "1": "Y position",
                "2": "X velocity",
                "3": "Y velocity",
                "4": "Angle",
                "5": "Angular velocity",
                "6": "Left contact sensor",
                "7": "Right contact sensor"
            },
            "min_values": [-1.5, -1.5, -5.0, -5.0, -3.14, -5.0, 0, 0],
            "max_values": [1.5, 1.5, 5.0, 5.0, 3.14, 5.0, 1, 1] 
        }, 
        "score": {
            "type": "number",
            "description": "The score received for the action."
        },  
        "completed": {
            "type": "boolean",
            "description": "Whether the landing event has ended (landing or accident)."
        }
    },
    "total score": {
        "type": "number",
        "description": "The total score received for the landing attempt."
    }
}

### UTILS

In [29]:
import json

def format_step_data(instant, action, next_state, reward, terminated):
    """ Format the step data into a JSON string
    
    Args:
        instant: int: the instant within the landing attempt where the current log is taken
        action: int: the action taken
        next_state: np.array: the next state of the environment
        reward: float: the reward received
        terminated: bool: whether the landing event has ended
        
    Returns:
        str: the step data formatted as a JSON string
    """
    # Convertir el array numpy a una lista
    next_state_list = next_state.tolist()

    # Redondear los elementos de la lista a 4 decimales
    next_state_list_rounded = [round(x, 3) for x in next_state_list]

    step_data = {
        'instant': instant,
        'action' : int(action),
        'current status': next_state_list_rounded,  # Convert numpy array to list
        'score': round(reward, 3),
        'completed': terminated,
        #'truncated': truncated
        #'info': info
    }
    
    #print(step_data)

    # Convert the dictionary to a JSON string
    step_data_json = json.dumps(step_data)

    return step_data_json


def format_episode_logs(logs, episode, total_score):
    """ Format the logs into a JSON string
    
    Args:
        logs: list: the logs for each step of the environment
        episode: int: the episode number
        
    Returns:
        str: the logs formatted as a JSON string
    """
    logs_data = {
        'landing attempt': episode,
        'logs': logs,
        'total score': total_score
    }

    # Convert the dictionary to a JSON string
    logs_json = json.dumps(logs_data)

    return logs_json

In [30]:
def commit_changes(repo_path, commit_message):
    """ Commit changes to the repository.

    Args:
        repo_path (str): Path to the repository.
        commit_message (str): The commit message.
    """
    repo = Repo(repo_path)
    repo.git.add(update=True)
    repo.index.commit(commit_message)
    origin = repo.remote(name='origin')
    origin.push()

In [31]:
import Action

def lunar_lander(max_t=1000, n_games=1, display=False, seed=38, agent=None):
    
    # Incluimos la opción del entorno gráfico.
    if display:
        env = gym.make('LunarLander-v2', render_mode='human')
    else:
        env = gym.make('LunarLander-v2')

    logs = []
    for episode in range(1, n_games+1):
        state = env.reset(seed=seed) # Set a seed for the environment
        state = state[0] # Eliminamos el diccionario vacio y dejamos unicamente el estado de 8 elementos.
        score = 0
        instant = 0
        episode_actions = []
        episode_rewards = []
        for i in range(max_t):
            if agent: 
                action = agent.act(state)
            else:
                action = Action.act(state)
                
            next_state, reward, terminated, truncated, info = env.step(action)
            json_logs = format_step_data(instant, action, next_state, reward, terminated)
            episode_rewards.append(reward)
            score += reward
            instant += 1
            if len(episode_actions) == 0 or len(episode_actions) % 3 == 0 or terminated:
                logs.append(json_logs)
            episode_actions.append(action)
            state = next_state
            if terminated:
                break
        json_episode_logs = format_episode_logs(logs, episode, score) 
        
        print(f"Número de instantes: {instant+1}. Tamaño de logs: {len(logs)}")
        print('episode ', episode, 'score %.3f' % float(score), 'avg score %.3f' % (float(score) / instant))
        
    env.close()
    return json_episode_logs

#### Logs del código inicial

In [32]:
initial_logs = lunar_lander(n_games=2, display=True)
initial_logs

Número de instantes: 107. Tamaño de logs: 36
episode  1 score -368.573 avg score -3.477
Número de instantes: 101. Tamaño de logs: 70
episode  2 score -175.130 avg score -1.751


'{"landing attempt": 2, "logs": ["{\\"instant\\": 0, \\"action\\": 1, \\"current status\\": [0.007, 1.417, 0.37, 0.131, -0.007, -0.044, 0.0, 0.0], \\"score\\": 1.387, \\"completed\\": false}", "{\\"instant\\": 3, \\"action\\": 0, \\"current status\\": [0.019, 1.425, 0.387, 0.098, -0.02, -0.086, 0.0, 0.0], \\"score\\": 0.083, \\"completed\\": false}", "{\\"instant\\": 6, \\"action\\": 0, \\"current status\\": [0.031, 1.43, 0.395, 0.066, -0.032, -0.079, 0.0, 0.0], \\"score\\": -0.026, \\"completed\\": false}", "{\\"instant\\": 9, \\"action\\": 3, \\"current status\\": [0.042, 1.432, 0.4, 0.015, -0.046, -0.126, 0.0, 0.0], \\"score\\": -1.515, \\"completed\\": false}", "{\\"instant\\": 12, \\"action\\": 0, \\"current status\\": [0.054, 1.429, 0.424, -0.066, -0.077, -0.221, 0.0, 0.0], \\"score\\": -1.304, \\"completed\\": false}", "{\\"instant\\": 15, \\"action\\": 2, \\"current status\\": [0.068, 1.427, 0.455, -0.033, -0.108, -0.204, 0.0, 0.0], \\"score\\": -2.5, \\"completed\\": false}", 

#### Logs exitosos
Utilizar otras semillas para que no memorice el entorno

In [34]:
import DuelingDQN

agent = DuelingDQN.Agent(num_observaciones=8, num_acciones=4, red_modelo=DuelingDQN.DuelingQNetwork, seed=0)
# Cargamos los pesos del agente entrenado.
agent.load_weights('checkpoint_Dueling.pth')

success_logs = lunar_lander(n_games=1, display=True, seed=42, agent=agent)
success_logs

Número de instantes: 294. Tamaño de logs: 99
episode  1 score 250.956 avg score 0.857


'{"landing attempt": 1, "logs": ["{\\"instant\\": 0, \\"action\\": 1, \\"current status\\": [0.004, 1.425, 0.22, 0.295, -0.003, -0.004, 0.0, 0.0], \\"score\\": 2.08, \\"completed\\": false}", "{\\"instant\\": 3, \\"action\\": 0, \\"current status\\": [0.011, 1.441, 0.22, 0.215, -0.003, -0.004, 0.0, 0.0], \\"score\\": 1.413, \\"completed\\": false}", "{\\"instant\\": 6, \\"action\\": 1, \\"current status\\": [0.018, 1.452, 0.211, 0.135, -0.002, 0.032, 0.0, 0.0], \\"score\\": 2.092, \\"completed\\": false}", "{\\"instant\\": 9, \\"action\\": 0, \\"current status\\": [0.024, 1.457, 0.211, 0.055, 0.003, 0.032, 0.0, 0.0], \\"score\\": 0.528, \\"completed\\": false}", "{\\"instant\\": 12, \\"action\\": 0, \\"current status\\": [0.03, 1.458, 0.202, -0.026, 0.011, 0.068, 0.0, 0.0], \\"score\\": -0.45, \\"completed\\": false}", "{\\"instant\\": 15, \\"action\\": 0, \\"current status\\": [0.036, 1.452, 0.202, -0.106, 0.021, 0.068, 0.0, 0.0], \\"score\\": -1.215, \\"completed\\": false}", "{\\"in

#### Bucle iterativo

In [35]:
import traceback, importlib
import Action

Action = importlib.reload(Action)

def create_and_run_llm_loop(Incial_msg, logger, model="gpt-3.5-turbo", num_iterations=10):
    
    agente = AssistantOpenAI(ARCLABKEY_OPENAI)

    # Crea un asistente
    asistente = agente.create_assistant(model=model, description=DESCRIPTION, instructions=INSTRUCTIONS, name=NAME, tools=TOOLS)
    
    # Crea un hilo
    hilo = agente.create_thread()
    
    # Añade un mensaje inicial al hilo.
    msg = agente.add_message(hilo.id, role="user", content=Incial_msg)

    for i in range(num_iterations):
        
        logger.info(f"Iteration: {i+1}")
        compiled = False

        while not compiled:
            # Ejecuta el hilo
            ejecucion = agente.run(hilo.id, asistente.id)
            response = agente.get_run(ejecucion.id, hilo.id)

            # Obtén la respuesta utilizando un bucle de espera a que la respuesta esté lista.
            while response.status not in ["completed", "failed", "requires_action"]:
                logger.info(f"Status: {ejecucion.status}")
                response = agente.get_run(ejecucion.id, hilo.id)
                time.sleep(20)

            #print(f"Status: {response.status}")
            logger.info(f"Status: {response.status}")
            if response.status == "completed":
                return response
            tool_call = response.required_action.submit_tool_outputs.tool_calls
            #code = tool_call.function.arguments[0]
            # Convierte el string a un diccionario
            code_dict = json.loads(tool_call[0].function.arguments)
            logger.info(f"Arguments: {code_dict}")
            # Obtén el código Python de la llamada a la herramienta
            code = code_dict["code"]
            filename = code_dict["filename"]  

            logger.info(f"\nCodigo generado:\n{code}")

            # Ejecuta el código Python
            try:
                #exec(code)
                store_code_in_file(code, filename)
                time.sleep(1) # Pequeño retraso para que el sistema operativo pueda reflejar los cambios en el archivo
                importlib.reload(Action)
                
                logs = lunar_lander(n_games=1, display=True)
                
                for call in tool_call:
                    agente.devolver_respuesta(response.id, hilo.id, tool_outputs=[{"tool_call_id": call.id, "output": "Run successful."}])            
                compiled = True
                logger.info(f"Compilación exitosa.")
                # logger.info(f"Partida: {ref.game.game_log()}")
                while response.status not in ["completed", "failed", "expired"]:
                    logger.info(f"Status: {response.status}")
                    response = agente.get_run(response.id, hilo.id)
                    time.sleep(20)
                
                msg = f"""These are the logs generated in the last execution{logs}. You have to improve the last code generated to maximize the reward. Please, make it more efficient, faster and more robust."""
                logger.info(msg)    
                agente.add_message(hilo.id, role="user", content=msg)
                
            except Exception as e:
                logger.exception("Error: %s", e)
                error_trace = traceback.format_exc()
                for call in tool_call:
                    agente.devolver_respuesta(response.id, hilo.id, tool_outputs=[{"tool_call_id": call.id, "output": "ERROR."}]) 
                logger.error(f"Error: {e}.")
                while response.status not in ["completed", "failed", "expired"]:
                    logger.info(f"Status: {response.status}")
                    response = agente.get_run(ejecucion.id, hilo.id)
                    time.sleep(30)
                msg = f"The code generated has an error. Please, try again. Error: {e}. Trace: {error_trace}"   
                logger.error(msg)   
                agente.add_message(hilo.id, role="assistant", content=msg)
    
    agente.mostrar_mensajes(hilo.id)  
    vaciar_agente(agente)
    
    logger.info("\nEjecución finalizada.\n\n")       

    return response


def vaciar_agente(agente):
    for assistant in agente.assistants:
        agente.delete_assistant(assistant)

    for thread in agente.threads:
        agente.delete_thread(thread)
        
    print("Asistente vaciado.")


#### Logger

In [36]:
def configura_log(nombre_archivo):
    """ 
    Configura el logger para que guarde los logs en un archivo y los muestre en la consola.
    
    Args:
        nombre_archivo: str Nombre del archivo donde se guardarán los logs.
        
    Returns:
        logger: logging.Logger Objeto logger configurado.
    """
    # Crear la carpeta logs si no existe
    if not os.path.exists('logs'):
        os.makedirs('logs')
    # Configura el logger
    logging.basicConfig(filename=f'logs/{nombre_archivo}', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger()
    
    # Añade un StreamHandler para mostrar los logs en la consola
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    console_handler.setFormatter(formatter)
    logger.addHandler(console_handler)
    
    return logger

def summarize_logs(logs):
    """ Summarize the logs of the Lunar Lander game
    
    Args:
        logs: dict: logs of the game
        
    Returns:
        str: summary of the logs
    """
    rewards = logs['rewards']
    actions = logs['actions']

    reward_summary = f"Average reward: {np.mean(rewards)}, Max reward: {np.max(rewards)}, Min reward: {np.min(rewards)}"
    action_summary = f"Action frequencies: {np.bincount(actions)}"

    return reward_summary + "\n" + action_summary

### Ejecución con el asistente

In [12]:
DESCRIPTION = "You are an expert programer in Pyhton. Your specialty is to generate the code responsible for making decisions about actions to be taken in various spacecraft landing environments.The objective is to land the spacecraft within a target zone in the shortest possible time and very gently. A scoring system is used to evaluate the landings, which must be maximized."
INSTRUCTIONS = f"""Your task is:
1. Analyze and reason about the results received in the last landing attempts.
2. Your goal is to be able to make the appropriate decision based on the results of previous iterations. You must code the decision making based on your reasoning in a Python function.
3. Based on the logs you should improve the code generated in the 'act' function in the last iteration, seeking to maximize the score received and generate a higher quality code.
4. Save the code of the act function in the file 'Action.py'.
5. Improve your results and correct any errors you may have generated in your last code if they exist.
"""
NAME = "Spacecraft Landing Master"

initial_msg = f"Take a deep breath and think step by step. This is the record of an example of a successful landing in this environment, but under other conditions: {success_logs}. This is the code of the initial function: {agent_initial_code} and these are the execution logs of one landing attempt: {initial_logs}. Analyze the results and improve the code."

logger = configura_log('Spacecraft_4o.log')
response = create_and_run_llm_loop(initial_msg, logger, model="gpt-4o", num_iterations=5)

print(response)

2024-06-06 18:25:22,500 - INFO - HTTP Request: POST https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2024-06-06 18:25:22,701 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2024-06-06 18:25:22,966 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/messages "HTTP/1.1 200 OK"
2024-06-06 18:25:23,007 - INFO - Iteration: 1
2024-06-06 18:25:23,485 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs "HTTP/1.1 200 OK"
2024-06-06 18:25:23,796 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_gbjordgUJkCZoFGEXWL1eDr2 "HTTP/1.1 200 OK"
2024-06-06 18:25:23,799 - INFO - Status: queued
2024-06-06 18:25:24,019 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_gbjordgUJkCZoFGEXWL1eDr2 "HTTP/1.1 200 OK"
2024-06-06 18:25:44,021 - INFO - Status: queued
2024-06-06 18:25:44,347

Número de instantes: 220. Tamaño de logs: 45
episode  1 score -866.398 avg score -3.956


2024-06-06 18:26:30,819 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_gbjordgUJkCZoFGEXWL1eDr2/submit_tool_outputs "HTTP/1.1 200 OK"
2024-06-06 18:26:30,820 - INFO - Compilación exitosa.
2024-06-06 18:26:30,821 - INFO - Status: requires_action
2024-06-06 18:26:31,042 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_gbjordgUJkCZoFGEXWL1eDr2 "HTTP/1.1 200 OK"
2024-06-06 18:26:51,046 - INFO - Status: in_progress
2024-06-06 18:26:51,356 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_gbjordgUJkCZoFGEXWL1eDr2 "HTTP/1.1 200 OK"
2024-06-06 18:27:11,360 - INFO - Status: in_progress
2024-06-06 18:27:11,624 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_gbjordgUJkCZoFGEXWL1eDr2 "HTTP/1.1 200 OK"
2024-06-06 18:27:31,631 - INFO - These are the logs generated in the last execution{"l

Número de instantes: 215. Tamaño de logs: 44
episode  1 score -812.713 avg score -3.798


2024-06-06 18:28:40,046 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_Yn1XPtcuGCDf7Ymst2RV5Ajo/submit_tool_outputs "HTTP/1.1 200 OK"
2024-06-06 18:28:40,048 - INFO - Compilación exitosa.
2024-06-06 18:28:40,048 - INFO - Status: requires_action
2024-06-06 18:28:40,286 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_Yn1XPtcuGCDf7Ymst2RV5Ajo "HTTP/1.1 200 OK"
2024-06-06 18:29:00,290 - INFO - Status: in_progress
2024-06-06 18:29:00,575 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_Yn1XPtcuGCDf7Ymst2RV5Ajo "HTTP/1.1 200 OK"
2024-06-06 18:29:20,579 - INFO - These are the logs generated in the last execution{"landing attempt": 1, "logs": ["{\"instant\": 0, \"action\": 2, \"current status\": [0.007, 1.418, 0.377, 0.172, -0.009, -0.089, 0.0, 0.0], \"score\": -1.45, \"completed\": false}", "{\"instant\": 5, \"action\": 2, \"current

...

Número de instantes: 77. Tamaño de logs: 16
episode  1 score -190.821 avg score -2.511


2024-06-06 18:32:58,774 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_Sd7kT4uSqhxokj4Ub1pZKIIF/submit_tool_outputs "HTTP/1.1 200 OK"
2024-06-06 18:32:58,775 - INFO - Compilación exitosa.
2024-06-06 18:32:58,776 - INFO - Status: requires_action
2024-06-06 18:33:10,201 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_Sd7kT4uSqhxokj4Ub1pZKIIF "HTTP/1.1 200 OK"
2024-06-06 18:33:30,203 - INFO - Status: in_progress
2024-06-06 18:33:30,484 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79/runs/run_Sd7kT4uSqhxokj4Ub1pZKIIF "HTTP/1.1 200 OK"
2024-06-06 18:33:50,489 - INFO - These are the logs generated in the last execution{"landing attempt": 1, "logs": ["{\"instant\": 0, \"action\": 1, \"current status\": [0.007, 1.417, 0.37, 0.131, -0.007, -0.044, 0.0, 0.0], \"score\": 1.387, \"completed\": false}", "{\"instant\": 5, \"action\": 1, \"current 

user: [TextContentBlock(text=Text(annotations=[], value='These are the logs generated in the last execution{"landing attempt": 1, "logs": ["{\\"instant\\": 0, \\"action\\": 1, \\"current status\\": [0.007, 1.417, 0.37, 0.131, -0.007, -0.044, 0.0, 0.0], \\"score\\": 1.387, \\"completed\\": false}", "{\\"instant\\": 5, \\"action\\": 1, \\"current status\\": [0.025, 1.423, 0.323, -0.003, 0.011, 0.146, 0.0, 0.0], \\"score\\": 0.221, \\"completed\\": false}", "{\\"instant\\": 10, \\"action\\": 0, \\"current status\\": [0.04, 1.414, 0.28, -0.138, 0.077, 0.316, 0.0, 0.0], \\"score\\": -2.357, \\"completed\\": false}", "{\\"instant\\": 15, \\"action\\": 0, \\"current status\\": [0.055, 1.389, 0.28, -0.271, 0.156, 0.315, 0.0, 0.0], \\"score\\": -2.787, \\"completed\\": false}", "{\\"instant\\": 20, \\"action\\": 0, \\"current status\\": [0.07, 1.35, 0.28, -0.405, 0.235, 0.315, 0.0, 0.0], \\"score\\": -2.856, \\"completed\\": false}", "{\\"instant\\": 25, \\"action\\": 0, \\"current status\\": [

2024-06-06 18:34:01,127 - INFO - HTTP Request: DELETE https://api.openai.com/v1/assistants/asst_ILogTF2ctEpZIqSQG24A5QPD "HTTP/1.1 200 OK"
2024-06-06 18:34:01,374 - INFO - HTTP Request: DELETE https://api.openai.com/v1/threads/thread_Qs60jCEyyuKq3EZ62guFcp79 "HTTP/1.1 200 OK"
2024-06-06 18:34:01,376 - INFO - 
Ejecución finalizada.




Asistente vaciado.
Run(id='run_Sd7kT4uSqhxokj4Ub1pZKIIF', assistant_id='asst_ILogTF2ctEpZIqSQG24A5QPD', cancelled_at=None, completed_at=1717691592, created_at=1717691533, expires_at=None, failed_at=None, incomplete_details=None, instructions="Your task is:\n1. Analyze and reason about the results received in the last landing attempts.\n2. Your goal is to be able to make the appropriate decision based on the results of previous iterations. You must code the decision making based on your reasoning in a Python function.\n3. Based on the logs you should improve the code generated in the 'act' function in the last iteration, seeking to maximize the score and generate a higher quality code.\n4. Save the code of the act function in the file 'Action.py'.\n5. Improve your results and correct any errors you may have generated in your last code if they exist.\n", last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', required_action=

In [13]:
import importlib
import Prueba

Prueba = importlib.reload(Prueba)
Prueba.prueba()


def funcion_prueba():
    for i in range(5):
    # 
        code = f"""
def prueba():
    print("Método modificado en la iteración {i}.")
        """
        #exec(code)
        store_code_in_file(code, "Prueba.py")
        time.sleep(1) # Pequeño retraso para que el sistema operativo pueda reflejar los cambios en el archivo.

        importlib.reload(Prueba)
        Prueba.prueba()
        

funcion_prueba()




Método modificado en la iteración 4.
Método modificado en la iteración 0.
Método modificado en la iteración 1.
Método modificado en la iteración 2.
Método modificado en la iteración 3.
Método modificado en la iteración 4.


## Tasks

* Incluir guardado de vídeo.
* Mejorar el prompt inicial.
* Incluir ejemplos positivos.
* Incluir cambios en el código con GIT.